# Predicting Proposed Treatment Type Using Logistic Regression

### Notebook automatically generated from your model

Model Logistic Regression (Prediction), trained on 2022-09-24 14:16:24.

#### Generated on 2022-09-24 23:30:51.997235

prediction
This notebook will reproduce the steps for a MULTICLASS on  BTM_data_prepared.
The main objective is to predict the variable Proposed Treatment Type (Insert/Injection; Manual Removal; Organic Spray)

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [1]:
import sys
import dataiku
import numpy as np
import pandas as pd
import sklearn as sk
import dataiku.core.pandasutils as pdu
from dataiku.doctor.preprocessing import PCA
from collections import defaultdict, Counter

And tune pandas display options:

In [2]:
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#### Importing base data

The first step is to get our machine learning dataset:

In [3]:
# We apply the preparation that you defined. You should not modify this.
preparation_steps = []
preparation_output_schema = {'columns': [{'name': 'Tree ID', 'type': 'string'}, {'name': 'Longitude', 'type': 'double'}, {'name': 'Latitude', 'type': 'double'}, {'name': 'Common Name', 'type': 'string'}, {'name': 'Condition Class / Infestation Pattern', 'type': 'string'}, {'name': 'List of Trees on  residential property', 'type': 'string'}, {'name': 'BTM positive trees self reported  on residential property', 'type': 'string'}, {'name': 'Number of BTM positive Tree Species on property', 'type': 'bigint'}, {'name': 'Dbh 1. inches', 'type': 'bigint'}, {'name': 'Dbh 2', 'type': 'bigint'}, {'name': 'Dbh 3', 'type': 'bigint'}, {'name': 'Dbh 4', 'type': 'bigint'}, {'name': 'Dbh 5', 'type': 'bigint'}, {'name': 'Dbh 6', 'type': 'bigint'}, {'name': 'Number Observed BTM Nests', 'type': 'bigint'}, {'name': 'Entry Source', 'type': 'string'}, {'name': 'BTM (Y/N)', 'type': 'boolean'}, {'name': 'Interest in Help', 'type': 'string'}, {'name': 'Tree Type (Ornamental | Fruiting / Flowering | Bush)', 'type': 'string'}, {'name': 'Distance to Water (feet)', 'type': 'string'}, {'name': 'Proposed Treatment Type (Insert/Injection; Manual Removal; Organic Spray)', 'type': 'string'}, {'name': 'Treatm.  Priority (1: High, 2: Med, 3: Low)', 'type': 'bigint'}, {'name': 'Geopoint', 'type': 'string'}], 'userModified': False}

ml_dataset_handle = dataiku.Dataset('BTM_data_prepared')
ml_dataset_handle.set_preparation_steps(preparation_steps, preparation_output_schema)
%time ml_dataset = ml_dataset_handle.get_dataframe(limit = 100000)

print ('Base data has %i rows and %i columns' % (ml_dataset.shape[0], ml_dataset.shape[1]))
# Five first records",
ml_dataset.head(5)

CPU times: user 21.3 ms, sys: 2.56 ms, total: 23.9 ms
Wall time: 67.9 ms
Base data has 932 rows and 23 columns


,Tree ID,Longitude,Latitude,Common Name,Condition Class / Infestation Pattern,List of Trees on residential property,BTM positive trees self reported on residential property,Number of BTM positive Tree Species on property,Dbh 1. inches,Dbh 2,Dbh 3,Dbh 4,Dbh 5,Dbh 6,Number Observed BTM Nests,Entry Source,BTM (Y/N),Interest in Help,Tree Type (Ornamental | Fruiting / Flowering | Bush),Distance to Water (feet),Proposed Treatment Type (Insert/Injection; Manual Removal; Organic Spray),"Treatm. Priority (1: High, 2: Med, 3: Low)",Geopoint
0,130.0,-69.633445,44.551271,Crabapple,Good,NaN,NaN,NaN,10,0,0,0,0,0,75,Bartlett Invent Feb 2022,Y,NaN,NaN,NaN,Organic Spray,1,POINT(-69.633445106 44.551270843)
1,101.0,-69.629734,44.549535,Crabapple,Fair,NaN,NaN,NaN,21,0,0,0,0,0,50,Bartlett Invent Feb 2022,Y,NaN,NaN,NaN,Organic Spray,1,POINT(-69.629734075 44.549535104)
2,59.0,-69.627131,44.551959,Elm-American,Good,NaN,NaN,NaN,4,0,0,0,0,0,50,Bartlett Invent Feb 2022,Y,NaN,NaN,NaN,Insert/Injection,1,POINT(-69.627130501 44.551959107)
3,105.0,-69.629361,44.549576,Elm-American,Good,NaN,NaN,NaN,42,0,0,0,0,0,50,Bartlett Invent Feb 2022,Y,NaN,NaN,NaN,Insert/Injection,1,POINT(-69.629360613 44.549576086)
4,353.0,-69.662301,44.528730,Oak-Northern Red,Good,NaN,NaN,NaN,24,0,0,0,0,0,50,Bartlett Invent Feb 2022,Y,NaN,NaN,NaN,Insert/Injection,1,POINT(-69.662300718 44.528730089)


#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

In [4]:
ml_dataset = ml_dataset[['Number Observed BTM Nests', 'Dbh 1. inches', 'Common Name', 'Latitude', 'BTM (Y/N)', 'Longitude', 'Distance to Water (feet)', 'List of Trees on  residential property', 'Condition Class / Infestation Pattern', 'Tree Type (Ornamental | Fruiting / Flowering | Bush)', 'Proposed Treatment Type (Insert/Injection; Manual Removal; Organic Spray)']]

Let's first coerce categorical columns into unicode, numerical features into floats.

In [5]:
# astype('unicode') does not work as expected

def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = ['Common Name', 'BTM (Y/N)', 'Distance to Water (feet)', 'List of Trees on  residential property', 'Condition Class / Infestation Pattern', 'Tree Type (Ornamental | Fruiting / Flowering | Bush)']
numerical_features = ['Number Observed BTM Nests', 'Dbh 1. inches', 'Latitude', 'Longitude']
text_features = []
from dataiku.doctor.utils import datetime_to_epoch
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    if ml_dataset[feature].dtype == np.dtype('M8[ns]') or (hasattr(ml_dataset[feature].dtype, 'base') and ml_dataset[feature].dtype.base == np.dtype('M8[ns]')):
        ml_dataset[feature] = datetime_to_epoch(ml_dataset[feature])
    else:
        ml_dataset[feature] = ml_dataset[feature].astype('double')

We are now going to handle the target variable and store it in a new variable:

In [6]:
target_map = {'Insert/Injection': 0, 'None': 1, 'Other': 2, 'Organic Spray': 3, 'Manual Removal': 4}
ml_dataset['__target__'] = ml_dataset['Proposed Treatment Type (Insert/Injection; Manual Removal; Organic Spray)'].map(str).map(target_map)
del ml_dataset['Proposed Treatment Type (Insert/Injection; Manual Removal; Organic Spray)']


# Remove rows for which the target is unknown.
ml_dataset = ml_dataset[~ml_dataset['__target__'].isnull()]

ml_dataset['__target__'] = ml_dataset['__target__'].astype(np.int64)

#### Cross-validation strategy

The dataset needs to be split into 2 new sets, one that will be used for training the model (train set)
and another that will be used to test its generalization capability (test set)

This is a simple cross-validation strategy.

In [7]:
train, test = pdu.split_train_valid(ml_dataset, prop=0.8)
print ('Train data has %i rows and %i columns' % (train.shape[0], train.shape[1]))
print ('Test data has %i rows and %i columns' % (test.shape[0], test.shape[1]))

Train data has 745 rows and 11 columns
Test data has 187 rows and 11 columns


#### Features preprocessing

The first thing to do at the features level is to handle the missing values.
Let's reuse the settings defined in the model

In [8]:
drop_rows_when_missing = []
impute_when_missing = [{'feature': 'Number Observed BTM Nests', 'impute_with': 'MEAN'}, {'feature': 'Dbh 1. inches', 'impute_with': 'MEAN'}, {'feature': 'Latitude', 'impute_with': 'MEAN'}, {'feature': 'Longitude', 'impute_with': 'MEAN'}]

# Features for which we drop rows with missing values"
for feature in drop_rows_when_missing:
    train = train[train[feature].notnull()]
    test = test[test[feature].notnull()]
    print ('Dropped missing records in %s' % feature)

# Features for which we impute missing values"
for feature in impute_when_missing:
    if feature['impute_with'] == 'MEAN':
        v = train[feature['feature']].mean()
    elif feature['impute_with'] == 'MEDIAN':
        v = train[feature['feature']].median()
    elif feature['impute_with'] == 'CREATE_CATEGORY':
        v = 'NULL_CATEGORY'
    elif feature['impute_with'] == 'MODE':
        v = train[feature['feature']].value_counts().index[0]
    elif feature['impute_with'] == 'CONSTANT':
        v = feature['value']
    train[feature['feature']] = train[feature['feature']].fillna(v)
    test[feature['feature']] = test[feature['feature']].fillna(v)
    print ('Imputed missing values in feature %s with value %s' % (feature['feature'], coerce_to_unicode(v)))

Imputed missing values in feature Number Observed BTM Nests with value 4.135570469798657
Imputed missing values in feature Dbh 1. inches with value 6.990604026845638
Imputed missing values in feature Latitude with value 44.55075008976781
Imputed missing values in feature Longitude with value -69.64718687628054


/opt/dss/dataiku-dss-10.0.5/python36.packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


We can now handle the categorical features (still using the settings defined in Models):

Let's dummy-encode the following features.
A binary column is created for each of the 100 most frequent values.

In [9]:
LIMIT_DUMMIES = 100

categorical_to_dummy_encode = ['Common Name', 'BTM (Y/N)', 'Distance to Water (feet)', 'List of Trees on  residential property', 'Condition Class / Infestation Pattern', 'Tree Type (Ornamental | Fruiting / Flowering | Bush)']

# Only keep the top 100 values
def select_dummy_values(train, features):
    dummy_values = {}
    for feature in categorical_to_dummy_encode:
        values = [
            value
            for (value, _) in Counter(train[feature]).most_common(LIMIT_DUMMIES)
        ]
        dummy_values[feature] = values
    return dummy_values

DUMMY_VALUES = select_dummy_values(train, categorical_to_dummy_encode)

def dummy_encode_dataframe(df):
    for (feature, dummy_values) in DUMMY_VALUES.items():
        for dummy_value in dummy_values:
            dummy_name = u'%s_value_%s' % (feature, coerce_to_unicode(dummy_value))
            df[dummy_name] = (df[feature] == dummy_value).astype(float)
        del df[feature]
        print ('Dummy-encoded feature %s' % feature)

dummy_encode_dataframe(train)

dummy_encode_dataframe(test)

Dummy-encoded feature Common Name
Dummy-encoded feature BTM (Y/N)
Dummy-encoded feature Distance to Water (feet)


/opt/dss/dataiku-dss-10.0.5/python36.packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Dummy-encoded feature List of Trees on  residential property
Dummy-encoded feature Condition Class / Infestation Pattern
Dummy-encoded feature Tree Type (Ornamental | Fruiting / Flowering | Bush)
Dummy-encoded feature Common Name
Dummy-encoded feature BTM (Y/N)
Dummy-encoded feature Distance to Water (feet)
Dummy-encoded feature List of Trees on  residential property
Dummy-encoded feature Condition Class / Infestation Pattern
Dummy-encoded feature Tree Type (Ornamental | Fruiting / Flowering | Bush)


Let's rescale numerical features

In [10]:
rescale_features = {'Number Observed BTM Nests': 'AVGSTD', 'Dbh 1. inches': 'AVGSTD', 'Latitude': 'AVGSTD', 'Longitude': 'AVGSTD'}
for (feature_name, rescale_method) in rescale_features.items():
    if rescale_method == 'MINMAX':
        _min = train[feature_name].min()
        _max = train[feature_name].max()
        scale = _max - _min
        shift = _min
    else:
        shift = train[feature_name].mean()
        scale = train[feature_name].std()
    if scale == 0.:
        del train[feature_name]
        del test[feature_name]
        print ('Feature %s was dropped because it has no variance' % feature_name)
    else:
        print ('Rescaled %s' % feature_name)
        train[feature_name] = (train[feature_name] - shift).astype(np.float64) / scale
        test[feature_name] = (test[feature_name] - shift).astype(np.float64) / scale

Rescaled Number Observed BTM Nests
Rescaled Dbh 1. inches
Rescaled Latitude
Rescaled Longitude


/opt/dss/dataiku-dss-10.0.5/python36.packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Modeling

Before actually creating our model, we need to split the datasets into their features and labels parts:

In [11]:
X_train = train.drop('__target__', axis=1)
X_test = test.drop('__target__', axis=1)

y_train = np.array(train['__target__'])
y_test = np.array(test['__target__'])

Now we can finally create our model!

In [12]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty="l2",random_state=1337)

We set "class_weight" as the weighting strategy:

In [13]:
clf.class_weight = "balanced"

... And train the model

In [14]:
%time clf.fit(X_train, y_train)

CPU times: user 29.1 ms, sys: 4.01 ms, total: 33.1 ms
Wall time: 31.1 ms


/opt/dss/dataiku-dss-10.0.5/python36.packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/dss/dataiku-dss-10.0.5/python36.packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=1337,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

Build up our result dataset

The model is now trained, we can apply it to our test set:

In [15]:
%time _predictions = clf.predict(X_test)
%time _probas = clf.predict_proba(X_test)
predictions = pd.Series(data=_predictions, index=X_test.index, name='predicted_value')
cols = [
    u'probability_of_value_%s' % label
    for (_, label) in sorted([(int(target_map[label]), label) for label in target_map])
]
probabilities = pd.DataFrame(data=_probas, index=X_test.index, columns=cols)

# Build scored dataset
results_test = X_test.join(predictions, how='left')
results_test = results_test.join(probabilities, how='left')
results_test = results_test.join(test['__target__'], how='left')
results_test = results_test.rename(columns= {'__target__': 'Proposed Treatment Type (Insert/Injection; Manual Removal; Organic Spray)'})

CPU times: user 2.29 ms, sys: 460 µs, total: 2.75 ms
Wall time: 1.86 ms
CPU times: user 2.51 ms, sys: 0 ns, total: 2.51 ms
Wall time: 1.71 ms


#### Results

You can measure the model's accuracy:

In [16]:
from dataiku.doctor.utils.metrics import mroc_auc_score
y_test_ser = pd.Series(y_test)
 
print ('AUC value:', mroc_auc_score(y_test_ser, _probas))

AUC value: 0.9844396344396344


We can also view the predictions directly.
Since scikit-learn only predicts numericals, the labels have been mapped to 0,1,2 ...
We need to 'reverse' the mapping to display the initial labels.

In [17]:
inv_map = { target_map[label] : label for label in target_map}
predictions.map(inv_map)

11     Insert/Injection
13     Insert/Injection
25     Insert/Injection
33       Manual Removal
34        Organic Spray
37     Insert/Injection
39     Insert/Injection
40     Insert/Injection
43     Insert/Injection
46     Insert/Injection
58     Insert/Injection
61     Insert/Injection
62     Insert/Injection
63     Insert/Injection
70     Insert/Injection
75        Organic Spray
79     Insert/Injection
82     Insert/Injection
107    Insert/Injection
108      Manual Removal
110    Insert/Injection
118    Insert/Injection
121    Insert/Injection
126    Insert/Injection
130    Insert/Injection
133    Insert/Injection
135    Insert/Injection
142       Organic Spray
145    Insert/Injection
147    Insert/Injection
151    Insert/Injection
154      Manual Removal
155      Manual Removal
159    Insert/Injection
160    Insert/Injection
167    Insert/Injection
171    Insert/Injection
176       Organic Spray
204       Organic Spray
210    Insert/Injection
211    Insert/Injection
214    Insert/In

That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
